In [2]:
import requests 
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm
import time
import re

In [3]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}

eng_url_head = 'https://search.etnews.com/etnews/search.php?category=CATEGORY1&kwd=kaist&pageNum='

eng_url_tail = '&pageSize=10&reSrchFlag=false&sort=1&startDate=&endDate=&sitegubun=&jisikgubun=&preKwd%5B0%5D=kaist#&pageNum=340'

url1 = []
# url 변화하는거 적당히 보고 알아서 잘 해야됨. 페이지 단위로 넘어가는게 아니라면 selenium 써야할 수도 있음.
for i in tqdm(range(340)):
    url_final = eng_url_head + str(i) +eng_url_tail
    try:
        response = requests.get(url_final, headers=headers)
        html = response.content
        soup = bs(html, "html.parser")
#            page가 넘어갔을 때 검색결과가 없는 경우 while 문 break
        elements = soup.select('.thumb a[href]') # F12 눌러서 적당히 우리가 원하는 a href 태그 url 링크 달려있는 id 또는 class 찾아서 넣어주면 됨.
    except:
        print('Request Failed_1')
    for element in elements:
        link = element.attrs['href']
        url1.append(link)

100%|████████████████████████████████████████████████████████████████████████████████| 340/340 [17:06<00:00,  3.02s/it]


In [4]:
url = list(set(url1))
df = pd.DataFrame({"url": url}, columns=["url"])
print("Total number of articles: ", len(df))
df.head()

Total number of articles:  3089


,url
0,//www.etnews.com/20160327000025
1,//www.etnews.com/20191015000210
2,//www.etnews.com/20180516000464
3,//www.etnews.com/20160216000163
4,//www.etnews.com/20170910000079


In [5]:
contents = []
titles = []
urls = []
for row in tqdm(df.itertuples()):
    link = 'https:' + getattr(row, 'url')
    response = requests.get(link, headers=headers)
    html = response.text
    soup = bs(html,'html.parser')
    try:
        for script in soup(["script", "style"]):
            script.decompose() 
        title = soup.select('.article_title')[0].get_text()
        content = soup.select('.article_body')[0].get_text() # select 안에 본문에 해당하는 id 값을 찾아서 넣어주면 됨.
        content = content.replace('\xa0', " ").replace('\u3000','').replace('\r','').replace('\n', " ")
        title = title.replace('\xa0', " ")
        title = title.replace('\n', " ")
        titles.append(title)
        contents.append(content)
        urls.append(link)
    except:
        print(row)
df = pd.DataFrame({"url": urls, "title": titles, "content": contents}, columns=["url", "title", "content"])
df.to_excel("articles/etnews.xlsx",index = False)
df.to_pickle("pickle files/etnews.pkl")

3089it [08:03,  6.39it/s]
